In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Carica il modello (stesso del tuo sistema)
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model = SentenceTransformer(MODEL_NAME)

# Test queries e risultati
test_cases = [
    {
        "query": "cosa dice l'articolo 50 del ccnl?",
        "result": """Art.50 – Sistemi di video sorveglianza aziendale"""
    },
    {
        "query": "cosa dice l'articolo 50 del ccnl?",
        "result": """Art.25 – Sistemi di video sorveglianza aziendale"""
    },
    {
        "query": "Posso assentarmi senza un preavviso?",
        "result": """Art.25 – Come richiedere le ferie"""
    }
]

print("="*80)
print("TEST SIMILARITY SENTENCE TRANSFORMER")
print("="*80)

for i, case in enumerate(test_cases, 1):
    query = case["query"]
    result = case["result"]
    
    # Genera embeddings
    query_embedding = model.encode([query])
    result_embedding = model.encode([result])
    
    # Calcola similarity
    similarity = cosine_similarity(query_embedding, result_embedding)[0][0]
    
    print(f"\nTest Case {i}:")
    print(f"Query: {query}")
    print(f"Result: {result}")
    print(f"Similarity Score: {similarity:.4f}")
    print("-" * 60)

TEST SIMILARITY SENTENCE TRANSFORMER

Test Case 1:
Query: cosa dice l'articolo 50 del ccnl?
Result: Art.50 – Sistemi di video sorveglianza aziendale
Similarity Score: 0.3548
------------------------------------------------------------

Test Case 2:
Query: cosa dice l'articolo 50 del ccnl?
Result: Art.25 – Sistemi di video sorveglianza aziendale
Similarity Score: 0.3621
------------------------------------------------------------

Test Case 3:
Query: Posso assentarmi senza un preavviso?
Result: Art.25 – Come richiedere le ferie
Similarity Score: 0.3851
------------------------------------------------------------


# Test Estrazione Titoli dal PDF CCNL

Utilizziamo le funzioni di `paragraph_chunker.py` per estrarre tutti i titoli dal PDF del CCNL commercio e testare la similarity con query specificate.

In [ ]:
# Estrazione titoli dal PDF CCNL usando paragraph_chunker.py
import sys
sys.path.append('.')

from paragraph_chunker import get_pdf_titles, extract_pages, remove_header

# Configurazione del PDF del CCNL
pdf_path = "datafile/ccnl_commercio_terziario_distribuzione_e_servizi.pdf"

print("="*80)
print("ESTRAZIONE TITOLI DAL PDF CCNL")
print("="*80)

# Estrai le pagine dal PDF
all_pages = extract_pages(pdf_path)
num_pages = len(all_pages)
print(f"Numero totale di pagine: {num_pages}")

# Rimuovi gli header comuni
final_pages, common_headers = remove_header(all_pages)
print(f"Header comuni rimossi: {len(common_headers)}")

# Estrai i titoli usando la funzione di paragraph_chunker
titles_per_page = get_pdf_titles(pdf_path, common_headers, num_pages, keep_page_title=False)

# Raccogli tutti i titoli unici (escludendo 'SKIP')
all_titles = []
for page_number, page_titles in enumerate(titles_per_page, start=1):
    if page_titles and page_titles != ['SKIP']:
        for title in page_titles:
            if title and title not in all_titles:
                all_titles.append(title)

print(f"\nTitoli unici estratti: {len(all_titles)}")
print("\nPrimi 20 titoli estratti:")
for i, title in enumerate(all_titles[:20], 1):
    print(f"{i:2d}. {title}")

print(f"\n... e altri {max(0, len(all_titles) - 20)} titoli")
print(f"\nTotale titoli disponibili per i test: {len(all_titles)}")

# Salva tutti i titoli in una variabile globale per i test successivi
extracted_titles = all_titles

/Users/francesco/Desktop/ACCA/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ESTRAZIONE TITOLI DAL PDF CCNL
Numero totale di pagine: 66
Header comuni rimossi: 3
Numero totale di pagine: 66
Header comuni rimossi: 3

Titoli unici estratti: 175

Primi 20 titoli estratti:
 1. VALIDITÀ DEL CONTRATTO
 2. INSCINDIBILITÀ
 3. PROCEDURE DI RINNOVO CONTRATTUALE
 4. DISTRIBUZIONE DEL CONTRATTO
 5. Stagionalità
 6. Art. 1 - Assunzione – requisiti per l’accesso
 7. Art.2 - Periodo di prova
 8. Art.3 - Mansioni lavorative e passaggi di livello
 9. Art.4 – Sicurezza sul luogo di lavoro, tutela della salute, formazione e rispetto dell’ambiente
10. Art.5 - Attuazione normativa D.LGS 81/08
11. Art.6 - Rappresentanti dei Lavoratori per la sicurezza-RLS/RLST
12. Art.7 - Articolazione dell’orario di lavoro
13. Art. 7 bis – Lavoratori Discontinui
14. Art. 7 ter – Reperibilità
15. Art.8 - Sospensione del lavoro e Recuperi
16. Art.9 - Flessibilità dell’orario contrattuale di lavoro-elasticità
17. Art.10 - Banca delle ore
18. Art.11 - Modalità di fruizione
19. Art.12 - Lavoro straordina

: 

## Test TF-IDF (1,3) - Top 5 Titoli Più Simili

Implementazione del test di similarity usando TF-IDF con n-gram (1,3) per trovare i 5 titoli più simili a una query specificata.

In [ ]:
# Test TF-IDF sui titoli estratti dal PDF CCNL
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("="*80)
print("TEST TF-IDF (1,3) - TITOLI ESTRATTI DAL PDF CCNL")
print("="*80)

# Query di test da specificare (stessa del test BM25 per confronto)
test_query = "Quali sono i diritti del lavoratore?"
test_query = "Posso assentarmi senza preavviso secondo il ccnl?"
test_query = "Cosa dice l'articolo 51 del ccnl?" #0.37
test_query = "Dimmi l'articolo 10 del ccnl" #0.29
test_query = "Cosa dice l'articolo 144 del ccnl?" # not found

# Verifica se abbiamo i titoli estratti
try:
    titles_to_test = extracted_titles
    print(f"Utilizzo {len(titles_to_test)} titoli estratti dal PDF")
except NameError:
    print("❌ Esegui prima la cella di estrazione titoli!")
    titles_to_test = []

if titles_to_test:
    # TF-IDF con n-gram (1,3) - include unigram, bigram e trigram
    vectorizer = TfidfVectorizer(ngram_range=(1,3), lowercase=True)
    
    # Fit del vectorizer sui titoli
    vectorizer.fit(titles_to_test)
    
    # Trasforma titoli e query nello stesso spazio vettoriale
    title_vectors = vectorizer.transform(titles_to_test)
    query_vector = vectorizer.transform([test_query])
    
    print(f"\n🔍 QUERY: {test_query}")
    print("-" * 70)
    
    # Calcola la cosine similarity tra query e tutti i titoli
    similarities = cosine_similarity(query_vector, title_vectors).flatten()
    
    # Ordina risultati per similarity decrescente
    ranked_results = list(zip(titles_to_test, similarities))
    ranked_results.sort(key=lambda x: x[1], reverse=True)
    
    # Mostra i TOP 5 risultati
    print(f"\n🎯 TOP 5 TITOLI PIÙ SIMILI (TF-IDF 1,3):")
    print("-" * 70)
    
    for i, (title, score) in enumerate(ranked_results[:5], 1):
        print(f"{i}. Score: {score:.4f}")
        print(f"   Titolo: {title}")
        print()
    
    # Statistiche aggiuntive
    print(f"📊 STATISTICHE:")
    print(f"   - Score massimo: {max(similarities):.4f}")
    print(f"   - Score minimo: {min(similarities):.4f}")
    print(f"   - Score medio: {np.mean(similarities):.4f}")
    print(f"   - Titoli con score > 0: {sum(1 for s in similarities if s > 0)}")
    
    # Info sui feature estratti
    feature_names = vectorizer.get_feature_names_out()
    print(f"   - Numero di feature (n-gram): {len(feature_names)}")
    print(f"   - Esempi di feature: {list(feature_names[:10])}")

print("\n" + "="*80)
print("✅ TF-IDF con n-gram (1,3) cattura sia parole singole che frasi!")
print("✅ Buon compromesso tra precisione lessicale e comprensione del contesto.")

TEST TF-IDF (1,3) - TITOLI ESTRATTI DAL PDF CCNL
Utilizzo 175 titoli estratti dal PDF

🔍 QUERY: Cosa dice l'articolo 144 del ccnl?
----------------------------------------------------------------------

🎯 TOP 5 TITOLI PIÙ SIMILI (TF-IDF 1,3):
----------------------------------------------------------------------
1. Score: 0.2611
   Titolo: VALIDITÀ DEL CONTRATTO

2. Score: 0.2611
   Titolo: DISTRIBUZIONE DEL CONTRATTO

3. Score: 0.2415
   Titolo: Art.102 – Tutela del lavoratore, esercizio del potere disciplinare

4. Score: 0.2209
   Titolo: Art.1 – Classificazione del personale

5. Score: 0.1920
   Titolo: Art.60 - Disciplina del rapporto

📊 STATISTICHE:
   - Score massimo: 0.2611
   - Score minimo: 0.0000
   - Score medio: 0.0208
   - Titoli con score > 0: 21
   - Numero di feature (n-gram): 1712
   - Esempi di feature: ['08', '10', '10 banca', '10 banca delle', '10 trattamento', '10 trattamento economico', '100', '100 contratto', '100 contratto di', '101']

✅ TF-IDF con n-gram (1,3) 

## Test BGE Reranker - Top 5 Titoli Più Simili

Implementazione del test di similarity usando BGE reranker per trovare i 5 titoli più simili a una query specificata.

In [ ]:
# Test BGE Reranker sui titoli estratti dal PDF CCNL
from sentence_transformers import CrossEncoder
import numpy as np

print("="*80)
print("TEST BGE RERANKER - TITOLI ESTRATTI DAL PDF CCNL")
print("="*80)

# BGE reranker multilingue, state-of-the-art per query-document matching
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=512)

# Query di test da specificare (stessa del test BM25 per confronto)
test_query = "cosa dice l'articolo 50 sulla videosorveglianza?"
test_query = "Posso assentarmi senza preavviso?" #0.04
test_query = "Cosa dice l'articolo 144 del ccnl?" #0.07
test_query = "Dimmi l'articolo 2 del ccnl" #0.01

# Verifica se abbiamo i titoli estratti
try:
    titles_to_test = extracted_titles
    print(f"Utilizzo {len(titles_to_test)} titoli estratti dal PDF")
except NameError:
    print("❌ Esegui prima la cella di estrazione titoli!")
    titles_to_test = []

if titles_to_test:
    print(f"\n🔍 QUERY: {test_query}")
    print("-" * 70)
    
    # Crea coppie query-title per reranking
    query_title_pairs = [[test_query, title] for title in titles_to_test]
    
    print(f"Elaborando {len(query_title_pairs)} coppie query-titolo...")
    
    # Calcola scores di reranking (output: logit score di rilevanza)
    # Nota: questo può richiedere del tempo per molti titoli
    scores = reranker.predict(query_title_pairs)
    
    # Ordina risultati per score decrescente
    ranked_results = list(zip(titles_to_test, scores))
    ranked_results.sort(key=lambda x: x[1], reverse=True)
    
    # Mostra i TOP 5 risultati
    print(f"\n🎯 TOP 5 TITOLI PIÙ SIMILI (BGE RERANKER):")
    print("-" * 70)
    
    for i, (title, score) in enumerate(ranked_results[:5], 1):
        print(f"{i}. Score: {score:.4f}")
        print(f"   Titolo: {title}")
        print()
    
    # Statistiche aggiuntive
    print(f"📊 STATISTICHE:")
    print(f"   - Score massimo: {max(scores):.4f}")
    print(f"   - Score minimo: {min(scores):.4f}")
    print(f"   - Score medio: {np.mean(scores):.4f}")
    print(f"   - Titoli con score > 0: {sum(1 for s in scores if s > 0)}")
    
    # Confronto con top results BM25 (se disponibili)
    print(f"\n🔄 CONFRONTO CON BM25:")
    print("   Esegui entrambi i test per vedere le differenze tra i metodi")

print("\n" + "="*80)
print("🎯 BGE Reranker è ottimizzato per comprensione semantica!")
print("🎯 Dovrebbe identificare meglio i match semantici rispetto a BM25.")

TEST BGE RERANKER - TITOLI ESTRATTI DAL PDF CCNL
Utilizzo 175 titoli estratti dal PDF

🔍 QUERY: Dimmi l'articolo 2 del ccnl
----------------------------------------------------------------------
Elaborando 175 coppie query-titolo...
Utilizzo 175 titoli estratti dal PDF

🔍 QUERY: Dimmi l'articolo 2 del ccnl
----------------------------------------------------------------------
Elaborando 175 coppie query-titolo...

🎯 TOP 5 TITOLI PIÙ SIMILI (BGE RERANKER):
----------------------------------------------------------------------
1. Score: 0.0170
   Titolo: Art.2 - Periodo di prova

2. Score: 0.0128
   Titolo: Art.2 - Contenuti della lettera di assunzione

3. Score: 0.0054
   Titolo: Art.102 – Tutela del lavoratore, esercizio del potere disciplinare

4. Score: 0.0052
   Titolo: Art.1 – Classificazione del personale

5. Score: 0.0037
   Titolo: Art.140 - Controversie collettive

📊 STATISTICHE:
   - Score massimo: 0.0170
   - Score minimo: 0.0000
   - Score medio: 0.0011
   - Titoli con score